<a href="https://colab.research.google.com/github/m109103/bert/blob/main/MyBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-bert

  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33517 sha256=53e68e8d1112a653afd556c5a84c7888ffd6a607b1861c107c40b6801aa6b95d
  Stored in directory: /root/.cache/pip/wheels/a4/e8/45/842b3a39831261aef9154b907eacbc4ac99499a99ae829b06f
  Created wheel for keras-transformer: filename=keras_transformer-0.40.0-py3-none-any.whl size=12305 sha256=3c7c4ef485f5a5a3b0d00f7d8a3ff08d0ece63fb48387b131bb1e3f4302c97fc
  Stored in directory: /root/.cache/pip/wheels/46/68/26/692ed21edd832833c3b0a0e21615bcacd99ca458b3f9ed571f
  Created wheel for keras-embed-sim: filename=keras_embed_sim-0.10.0-py3-none-any.whl size=3960 sha256=e5fe0d954449231c4fe605cd49bc7c698f202fae94202f5ff5ef18c23e247a4a
  Stored in directory: /root/.cache/pip/wheels/81/67/b5/d847588d075895281e1cf5590f819bd4cf076a554872268bd5
  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.16.0-py3-none-any.whl size=4668 sha256=8bfa813bc5410aae29271fc294d060d11325be3a6c968d803f1b4d28

In [2]:
!pip uninstall keras-nightly

In [3]:
!pip uninstall -y tensorflow

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Successfully uninstalled tensorflow-2.8.0


In [4]:
!pip install tensorflow-gpu==2.2.0

     |████████████████████████████████| 516.2 MB 4.8 kB/s 
     |████████████████████████████████| 3.0 MB 56.4 MB/s 
     |████████████████████████████████| 2.9 MB 38.7 MB/s 
     |████████████████████████████████| 454 kB 76.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3


In [5]:
!pip install keras==2.3.0

     |████████████████████████████████| 377 kB 21.1 MB/s 
     |████████████████████████████████| 50 kB 7.8 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0


In [6]:
!pip install keras-bert==0.82.0

  Created wheel for keras-bert: filename=keras_bert-0.82.0-py3-none-any.whl size=37462 sha256=757557db7bf783d55ab5b0521dd9b801b03e06a2ac390326257ca3b1c23e3fc5
  Stored in directory: /root/.cache/pip/wheels/24/41/78/49cff5ac891f83738a6b54486e830a0d54fbf62365cac358cb
Successfully built keras-bert
  Attempting uninstall: keras-bert
    Found existing installation: keras-bert 0.89.0
    Uninstalling keras-bert-0.89.0:
      Successfully uninstalled keras-bert-0.89.0


In [12]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [13]:
from google.colab import drive  
drive.mount('/content/gdrive')  

Mounted at /content/gdrive


In [11]:
!sudo lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.5 LTS
Release:	18.04
Codename:	bionic


In [8]:
import sys
import random
import numpy as np

from tensorflow.keras.layers import *
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

from keras_bert import (
    load_vocabulary,
    load_trained_model_from_checkpoint,
    Tokenizer,
    get_checkpoint_paths,
)
from keras_bert.datasets import get_pretrained, PretrainedList

Using TensorFlow backend.


In [9]:
SPLIT_PORTION = 0.8  # 指定訓練資料與測試資料的比例
BATCH_SIZE = 2  # batch size建議不要設得太大，不然很有可能out of memory
EPOCHS = 5 # epoch 5次其實就很夠了，當然你可以嘗試再大一點，只是訓練要更久

In [10]:
model_path = get_pretrained(PretrainedList.chinese_base)
paths = get_checkpoint_paths(model_path)
bert_model = load_trained_model_from_checkpoint(
    paths.config, paths.checkpoint, training=False, seq_len=None
)
bert_model.summary(line_length=120)

381894656/381892918 [==============================] - 5s 0us/step


AttributeError: ignored